In [ ]:
"Importing Libraries"
import numpy as np
import tensorflow as tf
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, LSTM, GRU, Dropout, Input, Layer, SimpleRNN
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from datetime import date, timedelta, datetime 
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from pandas.plotting import register_matplotlib_converters 
import matplotlib.dates as mdates 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from keras import Model
import time
import keras.backend as K
sns.set_style('white', { 'axes.spines.right': False, 'axes.spines.top': False})


In [2]:
import os
os.getcwd()
df = pd.read_excel("../Data/BTC Traditional Price Data (16_2_18 - 31_10_20).xlsx")
df = pd.DataFrame.from_records(df)
df.set_index('Date', inplace=True)
for col in df.columns:
    df[col] = df[col].astype('float32')

C:\Users\IraniaN\AppData\Local\Temp\ipykernel_9260\669066710.py:4: FutureWarning: Passing a DataFrame to DataFrame.from_records is deprecated. Use set_index and/or drop to modify the DataFrame instead.
  df = pd.DataFrame.from_records(df)


In [3]:
train_df = pd.DataFrame(df.copy())
train_df_ext = train_df.copy()
train_df_ext['Prediction'] = train_df_ext['BTC Price']
nrows = train_df.shape[0]
train_data_unscaled = np.array(train_df)
np_data = np.reshape(train_data_unscaled, (nrows, -1))
print("Train Data Set Shape: ", np_data.shape)

#Scaling
scaler = MinMaxScaler()
train_data_scaled = scaler.fit_transform(train_data_unscaled)
scaler_pred = MinMaxScaler()
df_Close = pd.DataFrame(train_df_ext['BTC Price'])
np_Close_scaled = scaler_pred.fit_transform(df_Close)

#Partitioning
sequence_length = 14
index_Close = train_df_ext.columns.get_loc("BTC Price")
train_data_len = math.ceil(train_data_scaled.shape[0] * 0.85)
train_data = train_data_scaled[0:train_data_len, :]
test_data = train_data_scaled[train_data_len - sequence_length:, :]

def partition_dataset(sequence_length, data):
    x, y = [], []
    data_len = data.shape[0]
    for i in range(sequence_length, data_len):
        x.append(data[i-sequence_length:i,:]) 
        y.append(data[i, index_Close])
    x = np.array(x)
    y = np.array(y)
    return x, y

x_train, y_train = partition_dataset(sequence_length, train_data)
x_test, y_test = partition_dataset(sequence_length, test_data)

print("X_train and y_train shapes: ", x_train.shape, y_train.shape)
print("x_test and y_test shapes: ", x_test.shape, y_test.shape)


Train Data Set Shape:  (989, 6)
X_train and y_train shapes:  (827, 14, 6) (827,)
x_test and y_test shapes:  (148, 14, 6) (148,)


In [4]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        e = K.tanh(K.dot(x,self.W)+self.b)
        e = K.squeeze(e, axis=-1)   
        alpha = K.softmax(e)
        alpha = K.expand_dims(alpha, axis=-1)
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

In [8]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer, LSTM, Dropout, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
import time

# Definition of attention layer
class Attention(Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], 1),
                                 initializer='random_normal', trainable=True)
        self.b = self.add_weight(name='attention_bias', shape=(input_shape[1], 1),
                                 initializer='zeros', trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x):
        # Calculation of attention weight for each time (time step)
        e = tf.nn.tanh(K.dot(x, self.W) + self.b)
        e = K.squeeze(e, axis=-1)  # vector with dimensions (batch_size, time_steps)
        alpha = K.softmax(e)  # Calculation of attention weight (normalization)
        alpha = K.expand_dims(alpha, axis=-1)  # convert to dimensions (batch_size, time_steps, 1)
        output = x * alpha # Apply attention weight to the input
        return output  # The output must be three-dimensional: (batch_size, time_steps, features)

    def compute_output_shape(self, input_shape):
        return input_shape  # Returns the same dimensions as the input

# Neural network model
model_input = Input(shape=(x_train.shape[1], x_train.shape[2]))  # ورودی مدل
x = LSTM(384, return_sequences=True, activation='tanh')(model_input)  # LSTM اول
attention_layer = Attention()(x)  # Apply attention layer
x = Dropout(0.3)(attention_layer)  # Dropout after attention

# A second LSTM that takes a 3D input
x = LSTM(384, return_sequences=False, activation='tanh')(x)  # LSTM دوم
x = Dense(1)(x)  # Dense layer for output

# Build the model
model = Model(model_input, x)

# Optimizer settings
adam = Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss='mean_squared_error')

# Early stop settings
early_stop = EarlyStopping(monitor='val_loss', mode="min", patience=40, verbose=1)

# Model training
start_time = time.time()
history = model.fit(
    x_train, y_train,
    batch_size=64,
    epochs=600,
    callbacks=[early_stop],
    validation_split=0.2,
    shuffle=False,
)

# Predict values
y_pred_scaled = model.predict(x_test)

# Execution time
print("--- %s seconds ---" % (time.time() - start_time))


Epoch 1/600
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 316ms/step - loss: 0.1606 - val_loss: 0.1745
Epoch 2/600
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 226ms/step - loss: 0.1079 - val_loss: 0.0704
Epoch 3/600
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 293ms/step - loss: 0.0337 - val_loss: 0.0181
Epoch 4/600
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 242ms/step - loss: 0.0174 - val_loss: 0.0079
Epoch 5/600
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 248ms/step - loss: 0.0082 - val_loss: 0.0075
Epoch 6/600
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 265ms/step - loss: 0.0074 - val_loss: 0.0079
Epoch 7/600
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 244ms/step - loss: 0.0069 - val_loss: 0.0074
Epoch 8/600
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 242ms/step - loss: 0.0063 - val_loss: 0.0077
Epoch 9/600
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 226ms/step - loss: 0.0063 - val_loss: 0.0076
Epoch 10/600
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 226ms/step - loss: 0.0062 - val_loss: 0.0076
Epoch 11/600
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 226ms/step - loss: 0.0063 - val_loss: 0.0078
Epoch 12/600
11/11 ━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
# Unscale the predicted values
y_pred = scaler_pred.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler_pred.inverse_transform(y_test.reshape(-1, 1))

r2 = r2_score(y_test_unscaled, y_pred)
print(f'R2: {np.round(r2, 4)}')

MAE = mean_absolute_error(y_test_unscaled, y_pred)
print(f'Median Absolute Error (MAE): {np.round(MAE, 2)}')

MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print(f'Mean Absolute Percentage Error (MAPE): {np.round(MAPE, 2)}')

R2: 0.927
Median Absolute Error (MAE): 223.00999450683594
Mean Absolute Percentage Error (MAPE): 2.0
